# 1. Tes Scraping di Halaman Utama

In [1]:
# Impor Library
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://www.mtsamples.com/index.asp'

In [3]:
page = requests.get(url)
page

<Response [200]>

In [4]:
soup = BeautifulSoup(page.text, 'html.parser')

In [5]:
links = soup.find_all('a', class_='dropdown-item')
for link in links[:40]:
    url = f"https://www.mtsamples.com/{link.get('href')} : {link.text}"
    print(url)

https://www.mtsamples.com//site/pages/browse.asp?Type=3-Allergy / Immunology : Allergy / Immunology
https://www.mtsamples.com//site/pages/browse.asp?Type=94-Autopsy : Autopsy
https://www.mtsamples.com//site/pages/browse.asp?Type=5-Bariatrics : Bariatrics
https://www.mtsamples.com//site/pages/browse.asp?Type=6-Cardiovascular / Pulmonary : Cardiovascular / Pulmonary
https://www.mtsamples.com//site/pages/browse.asp?Type=99-Chiropractic : Chiropractic
https://www.mtsamples.com//site/pages/browse.asp?Type=97-Consult - History and Phy. : Consult - History and Phy.
https://www.mtsamples.com//site/pages/browse.asp?Type=70-Cosmetic / Plastic Surgery : Cosmetic / Plastic Surgery
https://www.mtsamples.com//site/pages/browse.asp?Type=17-Dentistry : Dentistry
https://www.mtsamples.com//site/pages/browse.asp?Type=18-Dermatology : Dermatology
https://www.mtsamples.com//site/pages/browse.asp?Type=44-Diets and Nutritions : Diets and Nutritions
https://www.mtsamples.com//site/pages/browse.asp?Type=89-Di

# 2. Tes Scraping di Halaman Type & Specialty

In [6]:
type_url = f"https://www.mtsamples.com/{links[10].get('href')}"
type_page = requests.get(type_url)
records = BeautifulSoup(type_page.text, 'html.parser').find_all('td')
for record in records[:10]:
    print(f"{record.contents[1].text}:\nhttps://www.mtsamples.com/{record.contents[1].get('href')}\nDescription: {record.contents[-1]}")

Abscess with Cellulitis - Discharge Summary:
https://www.mtsamples.com//site/pages/sample.asp?Type=89-Discharge Summary&Sample=1254-Abscess with Cellulitis - Discharge Summary
Description: 
Incision and drainage, first metatarsal head, left foot with culture and sensitivity.

Anterior Cervical Discectomy & Fusion - Discharge Summary:
https://www.mtsamples.com//site/pages/sample.asp?Type=89-Discharge Summary&Sample=2520-Anterior Cervical Discectomy & Fusion - Discharge Summary
Description: 
Anterior cervical discectomy and fusions C4-5, C5-6, C6-7 using Bengal cages and Slimlock plate C4 to C7; intraoperative x-ray. Herniated nucleuses pulposus, C5-6 greater than C6-7, left greater than C4-5 right with left radiculopathy and moderate stenosis C5-6.

Back & Leg Pain - Discharge Summary:
https://www.mtsamples.com//site/pages/sample.asp?Type=89-Discharge Summary&Sample=2189-Back & Leg Pain - Discharge Summary
Description: 
Bilateral l5 spondylolysis with pars defects and spinal instability

In [7]:
records[0].contents[-1].replace('\n','')

'Incision and drainage, first metatarsal head, left foot with culture and sensitivity.'

# 3. Tes Scraping di Halaman Detail

In [8]:
detail_url = f"https://www.mtsamples.com/{records[10].contents[1].get('href')}"
detail_page = requests.get(detail_url)
detail_soup = BeautifulSoup(detail_page.text, 'html.parser')

In [9]:
# Mengambil Detail Text
raw_text = detail_soup.find('div', class_='hilightBold').contents[5:-15]
text = " ".join([str(i) for i in raw_text])
text

'<b>DIAGNOSIS AT ADMISSION: </b>  Congestive heart failure (CHF) with left pleural effusion. <br/> <br/> <b>DIAGNOSES AT DISCHARGE</b> <br/> 1. Congestive heart failure (CHF) with pleural effusion. <br/> 2. Hypertension. <br/> 3. Prostate cancer. <br/> 4. Leukocytosis. <br/> 5. Anemia of chronic disease. <br/> <br/> <b>HOSPITAL COURSE: </b> The patient was admitted to the emergency room by Dr. X. He has diuresed with IV Lasix. He was placed on Prinivil, aspirin, oxybutynin, docusate, and Klor-Con. Chest x-rays were followed. He did have free flowing fluid in his left chest. Radiology consultation was obtained for thoracentesis. The patient was seen by Dr. Y. An echocardiogram was done. This revealed an ejection fraction of 60% with diastolic dysfunction and periaortic stenosis with an opening of 1 cm3. An adenosine sestamibi was done in March 2000, with a small fixed apical defect, but no ischemia. Cardiac enzymes were negative. Dr. Y recommended a beta-blocker with an ACE inhibitor; t

In [10]:
# Membuat fungsi untuk memperbaiki Format Teks
import re
def cleantext(data):
    data = data.replace("(adsbygoogle = window.adsbygoogle || []).push({});","") # Menghapus ads google
    data = data.replace(u'\xa0', u'') # Menghapus Tab
    data = data.replace('\n', '') # Menghapus newline
    data = re.sub(r'<.*?>', '', data) # Menghapus tag html
    data = re.sub(' +',' ',data) # Mengubah Spasi Berlebih
    return data
print(cleantext(text))

DIAGNOSIS AT ADMISSION: Congestive heart failure (CHF) with left pleural effusion. DIAGNOSES AT DISCHARGE 1. Congestive heart failure (CHF) with pleural effusion. 2. Hypertension. 3. Prostate cancer. 4. Leukocytosis. 5. Anemia of chronic disease. HOSPITAL COURSE: The patient was admitted to the emergency room by Dr. X. He has diuresed with IV Lasix. He was placed on Prinivil, aspirin, oxybutynin, docusate, and Klor-Con. Chest x-rays were followed. He did have free flowing fluid in his left chest. Radiology consultation was obtained for thoracentesis. The patient was seen by Dr. Y. An echocardiogram was done. This revealed an ejection fraction of 60% with diastolic dysfunction and periaortic stenosis with an opening of 1 cm3. An adenosine sestamibi was done in March 2000, with a small fixed apical defect, but no ischemia. Cardiac enzymes were negative. Dr. Y recommended a beta-blocker with an ACE inhibitor; therefore, the lisinopril was discontinued. The patient felt much better after t

In [11]:
keywords = detail_soup.find('div', class_='mt-5 mb-2').contents[-1]
cleantext(keywords)

'discharge summary, prostate cancer, anemia of chronic disease, congestive heart failure, pleural effusion, thoracentesis, chf, anemia,'

# 4. Scraping Semua Records

In [15]:
# Membuat List-list kosong untuk menampung record
types_list = []
sample_list = []
description_list = []
transcript_list = []
keywords_list = []
# Looping ke setiap link yang ada di halaman utama
for link in links[:40]: 
    type_url = f"https://www.mtsamples.com/{link.get('href')}" # Mengambil link type/specialty
    type_page = requests.get(type_url)
    records = BeautifulSoup(type_page.text, 'html.parser').find_all('td') # Mengambil semua record sampel
    for record in records: # Looping pada setiap record yang ditemukan
        detail_url = f"https://www.mtsamples.com/{record.contents[1].get('href')}" # Mengambil link detail record
        detail_page = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_page.text, 'html.parser')
        raw_text = detail_soup.find('div', class_='hilightBold').contents[5:-15] # Mengambil data transkrip
        text = " ".join([str(i) for i in raw_text]) # Mengkonversi data transkrip dari list ke string
        keywords = detail_soup.find('div', class_='mt-5 mb-2').contents[-1] # Mengambil data keyword
        types_list.append(link.text) # Memasukkan nama type/specialty ke dalam list
        sample_list.append(record.contents[1].text) # Memasukkan nama sampel ke dalam list
        description_list.append(record.contents[-1].replace('\n','')) # Memasukkan deskripsi ke dalam list
        transcript_list.append(cleantext(text)) # Memasukkan transkrip ke dalam list
        keywords_list.append(cleantext(keywords)) # Memasukkan keyword ke dalam list
        print(f"Mengambil data ke {len(types_list)}, Subjudul: {link.text}\nSampel:{record.contents[1].text}\n")

Mengambil data ke 1, Subjudul: Allergy / Immunology
Sampel:Allergic Rhinitis

Mengambil data ke 2, Subjudul: Allergy / Immunology
Sampel:Allergy Evaluation Consult

Mengambil data ke 3, Subjudul: Allergy / Immunology
Sampel:Asthma in a 5-year-old

Mengambil data ke 4, Subjudul: Allergy / Immunology
Sampel:Chronic Sinusitis

Mengambil data ke 5, Subjudul: Allergy / Immunology
Sampel:Evaluation of Allergies

Mengambil data ke 6, Subjudul: Allergy / Immunology
Sampel:Followup on Asthma

Mengambil data ke 7, Subjudul: Allergy / Immunology
Sampel:Kawasaki Disease - Discharge Summary

Mengambil data ke 8, Subjudul: Autopsy
Sampel:Autopsy - 1

Mengambil data ke 9, Subjudul: Autopsy
Sampel:Autopsy - 2

Mengambil data ke 10, Subjudul: Autopsy
Sampel:Autopsy - 3

Mengambil data ke 11, Subjudul: Autopsy
Sampel:Autopsy - 4

Mengambil data ke 12, Subjudul: Autopsy
Sampel:Autopsy - 5

Mengambil data ke 13, Subjudul: Autopsy
Sampel:Autopsy - 6

Mengambil data ke 14, Subjudul: Autopsy
Sampel:Autopsy -

Mengambil data ke 96, Subjudul: Cardiovascular / Pulmonary
Sampel:Cardiac Catheterization - 12

Mengambil data ke 97, Subjudul: Cardiovascular / Pulmonary
Sampel:Cardiac Catheterization - 2

Mengambil data ke 98, Subjudul: Cardiovascular / Pulmonary
Sampel:Cardiac Catheterization - 3

Mengambil data ke 99, Subjudul: Cardiovascular / Pulmonary
Sampel:Cardiac Catheterization - 4

Mengambil data ke 100, Subjudul: Cardiovascular / Pulmonary
Sampel:Cardiac Catheterization - 5

Mengambil data ke 101, Subjudul: Cardiovascular / Pulmonary
Sampel:Cardiac Catheterization - 6

Mengambil data ke 102, Subjudul: Cardiovascular / Pulmonary
Sampel:Cardiac Catheterization - 7

Mengambil data ke 103, Subjudul: Cardiovascular / Pulmonary
Sampel:Cardiac Catheterization - 8

Mengambil data ke 104, Subjudul: Cardiovascular / Pulmonary
Sampel:Cardiac Catheterization - 9

Mengambil data ke 105, Subjudul: Cardiovascular / Pulmonary
Sampel:Cardiac Consult & Cardioversion

Mengambil data ke 106, Subjudul: Cardio

Mengambil data ke 183, Subjudul: Cardiovascular / Pulmonary
Sampel:Discharge Summary

Mengambil data ke 184, Subjudul: Cardiovascular / Pulmonary
Sampel:Discharge Summary - COPD

Mengambil data ke 185, Subjudul: Cardiovascular / Pulmonary
Sampel:Discharge Summary - Respiratory Distress

Mengambil data ke 186, Subjudul: Cardiovascular / Pulmonary
Sampel:Discharge Summary - Respiratory Failure

Mengambil data ke 187, Subjudul: Cardiovascular / Pulmonary
Sampel:Dobutamine Stress Test

Mengambil data ke 188, Subjudul: Cardiovascular / Pulmonary
Sampel:Dobutamine Stress Test - 1

Mengambil data ke 189, Subjudul: Cardiovascular / Pulmonary
Sampel:Double Lumen Port Inserstion

Mengambil data ke 190, Subjudul: Cardiovascular / Pulmonary
Sampel:Dual Chamber Generator Replacement

Mengambil data ke 191, Subjudul: Cardiovascular / Pulmonary
Sampel:Dual Chamber ICD Implantation

Mengambil data ke 192, Subjudul: Cardiovascular / Pulmonary
Sampel:Duplex Ultrasound - Legs

Mengambil data ke 193, Subj

Mengambil data ke 265, Subjudul: Cardiovascular / Pulmonary
Sampel:Lobectomy & Lymphadenectomy

Mengambil data ke 266, Subjudul: Cardiovascular / Pulmonary
Sampel:Lower Extremity Arterial Doppler

Mengambil data ke 267, Subjudul: Cardiovascular / Pulmonary
Sampel:Lower Extremity Venous Doppler

Mengambil data ke 268, Subjudul: Cardiovascular / Pulmonary
Sampel:Lung adenocarcinoma Path Report

Mengambil data ke 269, Subjudul: Cardiovascular / Pulmonary
Sampel:Lung Biopsy Discussion

Mengambil data ke 270, Subjudul: Cardiovascular / Pulmonary
Sampel:Lung Biopsy Pathology Report

Mengambil data ke 271, Subjudul: Cardiovascular / Pulmonary
Sampel:Lung Biopsy Pathology Report - 1

Mengambil data ke 272, Subjudul: Cardiovascular / Pulmonary
Sampel:Lung Cancer & MI - Hospice Cosult

Mengambil data ke 273, Subjudul: Cardiovascular / Pulmonary
Sampel:Lymphoblastic Leukemia - Consult

Mengambil data ke 274, Subjudul: Cardiovascular / Pulmonary
Sampel:Mayoview

Mengambil data ke 275, Subjudul: Ca

Mengambil data ke 352, Subjudul: Cardiovascular / Pulmonary
Sampel:Stress Test Bruce Protocol

Mengambil data ke 353, Subjudul: Cardiovascular / Pulmonary
Sampel:Stress Test Dobutamine

Mengambil data ke 354, Subjudul: Cardiovascular / Pulmonary
Sampel:Stress Test Dobutamine Myoview

Mengambil data ke 355, Subjudul: Cardiovascular / Pulmonary
Sampel:Stress Test Dobutrex

Mengambil data ke 356, Subjudul: Cardiovascular / Pulmonary
Sampel:Stress Test Graded Exercise Treadmill

Mengambil data ke 357, Subjudul: Cardiovascular / Pulmonary
Sampel:Stress Test Thallium

Mengambil data ke 358, Subjudul: Cardiovascular / Pulmonary
Sampel:Subclavian Central Venous Catheter Insertion

Mengambil data ke 359, Subjudul: Cardiovascular / Pulmonary
Sampel:Subxiphoid Pericardial Window

Mengambil data ke 360, Subjudul: Cardiovascular / Pulmonary
Sampel:Subxiphoid Pericardial Window - 1

Mengambil data ke 361, Subjudul: Cardiovascular / Pulmonary
Sampel:Subxiphoid Pericardiotomy

Mengambil data ke 362, S

...

Mengambil data ke 4553, Subjudul: Surgery
Sampel:Paracentesis

Mengambil data ke 4554, Subjudul: Surgery
Sampel:Paracentesis - Ultrasound-Guided

Mengambil data ke 4555, Subjudul: Surgery
Sampel:Paraphimosis

Mengambil data ke 4556, Subjudul: Surgery
Sampel:Parathyroid Adenoma Excision

Mengambil data ke 4557, Subjudul: Surgery
Sampel:Parotidectomy

Mengambil data ke 4558, Subjudul: Surgery
Sampel:Pars Plana Vitrectomy & Lensectomy

Mengambil data ke 4559, Subjudul: Surgery
Sampel:Patellar Tendon & Retinaculum Repair

Mengambil data ke 4560, Subjudul: Surgery
Sampel:Patent Ductus Arteriosus

Mengambil data ke 4561, Subjudul: Surgery
Sampel:Patent Ductus Arteriosus Ligation

Mengambil data ke 4562, Subjudul: Surgery
Sampel:Pectoralis Tendon Repair

Mengambil data ke 4563, Subjudul: Surgery
Sampel:PEG Tube

Mengambil data ke 4564, Subjudul: Surgery
Sampel:Pelvic Laparotomy

Mengambil data ke 4565, Subjudul: Surgery
Sampel:Penile Prosthesis Replacement

Mengambil data ke 4566, Subjudul: S

Mengambil data ke 4658, Subjudul: Surgery
Sampel:Salvage Cystectomy

Mengambil data ke 4659, Subjudul: Surgery
Sampel:Saphenous Vein - Ligation & Stripping

Mengambil data ke 4660, Subjudul: Surgery
Sampel:Scalp Mole Skin Biopsy

Mengambil data ke 4661, Subjudul: Surgery
Sampel:Scarf Bunionectomy

Mengambil data ke 4662, Subjudul: Surgery
Sampel:Scleral Buckle Opening

Mengambil data ke 4663, Subjudul: Surgery
Sampel:Scleral Buckle Opening - General Anesthesia

Mengambil data ke 4664, Subjudul: Surgery
Sampel:Scleral Buckle Opening - Local Anesthesia

Mengambil data ke 4665, Subjudul: Surgery
Sampel:Scott Cannula

Mengambil data ke 4666, Subjudul: Surgery
Sampel:Scrotal Exploration

Mengambil data ke 4667, Subjudul: Surgery
Sampel:Sebaceous Cyst Excision

Mengambil data ke 4668, Subjudul: Surgery
Sampel:Sebaceous Cyst Removal

Mengambil data ke 4669, Subjudul: Surgery
Sampel:Selective Coronary Angiography & Angioplasty

Mengambil data ke 4670, Subjudul: Surgery
Sampel:Septal Defect Rep

Mengambil data ke 4766, Subjudul: Surgery
Sampel:Total Hip Replacement 

Mengambil data ke 4767, Subjudul: Surgery
Sampel:Total Hip Replacement - 1

Mengambil data ke 4768, Subjudul: Surgery
Sampel:Total Knee Arthoplasty - Right - 1

Mengambil data ke 4769, Subjudul: Surgery
Sampel:Total Knee Arthroplasty

Mengambil data ke 4770, Subjudul: Surgery
Sampel:Total Knee Arthroplasty - Right

Mengambil data ke 4771, Subjudul: Surgery
Sampel:Total Knee Replacement

Mengambil data ke 4772, Subjudul: Surgery
Sampel:Total Knee Replacement - 1

Mengambil data ke 4773, Subjudul: Surgery
Sampel:Total Knee Replacement - NexGen

Mengambil data ke 4774, Subjudul: Surgery
Sampel:Total Thyroid Lumpectomy

Mengambil data ke 4775, Subjudul: Surgery
Sampel:Total Thyroidectomy

Mengambil data ke 4776, Subjudul: Surgery
Sampel:Trabeculectomy

Mengambil data ke 4777, Subjudul: Surgery
Sampel:Trabeculectomy & Tenonectomy

Mengambil data ke 4778, Subjudul: Surgery
Sampel:Tracheostomy

Mengambil data ke 4779, Su

Mengambil data ke 4877, Subjudul: Urology
Sampel:Cystoscopy

Mengambil data ke 4878, Subjudul: Urology
Sampel:Cystoscopy & Bladder Biopsy

Mengambil data ke 4879, Subjudul: Urology
Sampel:Cystoscopy & TURP

Mengambil data ke 4880, Subjudul: Urology
Sampel:Cystoscopy & Visual Urethrotomy

Mengambil data ke 4881, Subjudul: Urology
Sampel:Cystoscopy, Ureteropyelogram, & Ureteral Barbotage

Mengambil data ke 4882, Subjudul: Urology
Sampel:Cystourethroscopy & Retrograde Pyelogram

Mengambil data ke 4883, Subjudul: Urology
Sampel:Cystourethroscopy & Retrograde Pyelogram - 1

Mengambil data ke 4884, Subjudul: Urology
Sampel:Cystourethroscopy & TURP

Mengambil data ke 4885, Subjudul: Urology
Sampel:Cystourethroscopy & TURP - 1

Mengambil data ke 4886, Subjudul: Urology
Sampel:Cystourethroscopy & Urethral Dilation

Mengambil data ke 4887, Subjudul: Urology
Sampel:Discharge Summary - Urology

Mengambil data ke 4888, Subjudul: Urology
Sampel:Discharge Summary - Urology & Nephrology 

Mengambil da

Mengambil data ke 4985, Subjudul: Urology
Sampel:TURBT

Mengambil data ke 4986, Subjudul: Urology
Sampel:TURBT - 1

Mengambil data ke 4987, Subjudul: Urology
Sampel:TURP

Mengambil data ke 4988, Subjudul: Urology
Sampel:Ultrasound Scrotum

Mengambil data ke 4989, Subjudul: Urology
Sampel:Umbilical Hernia Repair

Mengambil data ke 4990, Subjudul: Urology
Sampel:Umbilical Hernia Repair - 1

Mengambil data ke 4991, Subjudul: Urology
Sampel:Ureteral Calculus - Consult

Mengambil data ke 4992, Subjudul: Urology
Sampel:Urinary Frequency & Urgency - Followup

Mengambil data ke 4993, Subjudul: Urology
Sampel:Urinary Retention

Mengambil data ke 4994, Subjudul: Urology
Sampel:Urinary Retention - Followup

Mengambil data ke 4995, Subjudul: Urology
Sampel:Urology Consut - 1

Mengambil data ke 4996, Subjudul: Urology
Sampel:Urology Discharge Summary

Mengambil data ke 4997, Subjudul: Urology
Sampel:Urology Letter

Mengambil data ke 4998, Subjudul: Urology
Sampel:Vasectomy

Mengambil data ke 4999, 

In [16]:
df = pd.DataFrame({"Type/Specialty":types_list,
                   "Sample Name":sample_list,
                   "Description":description_list,
                   "Transcript":transcript_list,
                   "Keywords":keywords_list})
df

,Type/Specialty,Sample Name,Description,Transcript,Keywords
0,Allergy / Immunology,Allergic Rhinitis,A 23-year-old white female presents with compl...,SUBJECTIVE: This 23-year-old white female pres...,"allergy / immunology, allergic rhinitis, aller..."
1,Allergy / Immunology,Allergy Evaluation Consult,"Acute allergic reaction, etiology uncertain, h...",HISTORY: A 34-year-old male presents today sel...,"allergy / immunology, keflex, acute allergic r..."
2,Allergy / Immunology,Asthma in a 5-year-old,Mother states he has been wheezing and coughing.,CHIEF COMPLAINT: This 5-year-old male presents...,"allergy / immunology, breathing treatment, air..."
3,Allergy / Immunology,Chronic Sinusitis,Patient having severe sinusitis about two to t...,HISTORY: I had the pleasure of meeting and eva...,"allergy / immunology, nasal congestion, facial..."
4,Allergy / Immunology,Evaluation of Allergies,"Chronic glossitis, xerostomia, probable enviro...",HISTORY: A 55-year-old female presents self-re...,"allergy / immunology, chronic glossitis, xeros..."
...,...,...,...,...,...
4998,Urology,Vasectomy - 1,Normal vasectomy,DESCRIPTION: The patient was placed in the sup...,"urology, vasectomy, allis clamp, catgut, hemoc..."
4999,Urology,Vasectomy - 2,Normal vasectomy,DIAGNOSIS: Desires vasectomy. NAME OF OPERATIO...,"urology, scrotal incision, right vas, bleeding..."
5000,Urology,Vasectomy - 3,Desire for sterility. Vasectomy. The vas was i...,PREOPERATIVE DIAGNOSIS: Desire for sterility. ...,"urology, vas, contralateral, desire for steril..."
5001,Urology,Vasectomy - 4,Fertile male with completed family. Elective m...,PROCEDURE: Elective male sterilization via bil...,"urology, sterilization, vas, fertile male, bil..."


In [17]:
df.shape

(5003, 5)

In [18]:
df.to_csv('mtsamples.csv', index=False)